# The Capstone Project - The Battle of Neighborhoods

## How to Choose the Best Berlin Borough to Open a Restaurant

**Business Problem Section**

*Introduction*:

Berlin is the Capital of Germany and has the greatest population (3,769,495) which is more than twice the second-most populace city, Hamburg.  Over the past 10 years, Berlin has grown at a rate of over 7%, more than any other city in Germany.  Investment is flowing into Berlin and new businesses are sprouting up across the Boroughs to meet the demand as the face of these neighborhoods change.   

Since Reunification in 1990 Berlin has had its ups and downs as many residents left for the West and initial investment floundered in the middle 1990s, leaving many real estate projects limbo.  Berlin embraced its artistic and creative potential during these years drawing creative-types to the city with its opportunities and very low rents. 

*Business Section*:

A family will be relocating from North America to Berlin in order to open a restaurant in one of the neighborhoods. The right location to open up is crucial for this family, so they need data to support their decision. Foursquare location data is a prime source to help this family solve its problems and make the best solution. The family will use the Foursquare data to find areas where the population and competition are at such levels that make it most conducive to open a restaurant in that location. 

**Data Section**

We will use only the foursquare data and a data frame retrieved from Wikipedia.org to solve this problem. We will first retrieve the data frame and clean it to show population, density, and import coordinates into the data frame. We will then import geolocator data to create a map of the Boroughs of Berlin. Next, we import Foursquare data regarding venues in each Borough including: closest, most common, frequency, top ten, and the mean of all types of venues in relation to each Borough. This data will give the family the necessary information to solve where to place their restaurant in conjunction with certain other outside data sources.

**Methodology**

In this part, I will present 3 main components of the report:

1, Data Collection: load all the required information into one single data frame.

2, Data Exploration and Understanding: inspect data, clean, extract and analyze all the hidden insights of Data Modeling.

3, K-means Clustering: cluster Berlin and nearby businesses, with focus on eateries to see how they group together and reach the final purpose to provide useful accommodation strategies.





**Results Section**

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

!conda install -c conda-forge wikipedia --yes 
import wikipedia as wp

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [3]:
html="https://en.wikipedia.org/wiki/Boroughs_and_neighborhoods_of_Berlin"

In [4]:
df = pd.read_html(html,header=0)[0]
df

,Borough,Population 31 March 2010,Area in km²,Density per km²,Map
0,Charlottenburg-Wilmersdorf,319628,64.72,4878,NaN
1,Friedrichshain-Kreuzberg,268225,20.16,13187,NaN
2,Lichtenberg,259881,52.29,4952,NaN
3,Marzahn-Hellersdorf,248264,61.74,4046,NaN
4,Mitte,332919,39.47,8272,NaN
5,Neukölln,310283,44.93,6804,NaN
6,Pankow,366441,103.01,3476,NaN
7,Reinickendorf,240454,89.46,2712,NaN
8,Spandau,223962,91.91,2441,NaN
9,Steglitz-Zehlendorf,293989,102.50,2818,NaN


In [5]:
df.rename(columns={'Population 31 March 2010': 'Population'}, inplace=True)
df

,Borough,Population,Area in km²,Density per km²,Map
0,Charlottenburg-Wilmersdorf,319628,64.72,4878,NaN
1,Friedrichshain-Kreuzberg,268225,20.16,13187,NaN
2,Lichtenberg,259881,52.29,4952,NaN
3,Marzahn-Hellersdorf,248264,61.74,4046,NaN
4,Mitte,332919,39.47,8272,NaN
5,Neukölln,310283,44.93,6804,NaN
6,Pankow,366441,103.01,3476,NaN
7,Reinickendorf,240454,89.46,2712,NaN
8,Spandau,223962,91.91,2441,NaN
9,Steglitz-Zehlendorf,293989,102.50,2818,NaN


In [6]:
df = df.drop(['Map'], axis = 1)
df

,Borough,Population,Area in km²,Density per km²
0,Charlottenburg-Wilmersdorf,319628,64.72,4878
1,Friedrichshain-Kreuzberg,268225,20.16,13187
2,Lichtenberg,259881,52.29,4952
3,Marzahn-Hellersdorf,248264,61.74,4046
4,Mitte,332919,39.47,8272
5,Neukölln,310283,44.93,6804
6,Pankow,366441,103.01,3476
7,Reinickendorf,240454,89.46,2712
8,Spandau,223962,91.91,2441
9,Steglitz-Zehlendorf,293989,102.50,2818


In [7]:
df.shape[0]

12

In [8]:
df['Latitude'] = [52.5, 52.5, 52.533333, 52.533333, 52.516667, 52.483333, 52.566667, 52.566667, 52.55, 52.433333, 52.466667, 52.45]
df

,Borough,Population,Area in km²,Density per km²,Latitude
0,Charlottenburg-Wilmersdorf,319628,64.72,4878,52.500000
1,Friedrichshain-Kreuzberg,268225,20.16,13187,52.500000
2,Lichtenberg,259881,52.29,4952,52.533333
3,Marzahn-Hellersdorf,248264,61.74,4046,52.533333
4,Mitte,332919,39.47,8272,52.516667
5,Neukölln,310283,44.93,6804,52.483333
6,Pankow,366441,103.01,3476,52.566667
7,Reinickendorf,240454,89.46,2712,52.566667
8,Spandau,223962,91.91,2441,52.550000
9,Steglitz-Zehlendorf,293989,102.50,2818,52.433333


In [9]:
df['Longitude'] = [13.283333, 13.45, 13.5, 13.583333,13.366667, 13.45, 13.4, 13.333333, 13.2, 13.25, 13.383333, 13.566667]

In [10]:
df

,Borough,Population,Area in km²,Density per km²,Latitude,Longitude
0,Charlottenburg-Wilmersdorf,319628,64.72,4878,52.500000,13.283333
1,Friedrichshain-Kreuzberg,268225,20.16,13187,52.500000,13.450000
2,Lichtenberg,259881,52.29,4952,52.533333,13.500000
3,Marzahn-Hellersdorf,248264,61.74,4046,52.533333,13.583333
4,Mitte,332919,39.47,8272,52.516667,13.366667
5,Neukölln,310283,44.93,6804,52.483333,13.450000
6,Pankow,366441,103.01,3476,52.566667,13.400000
7,Reinickendorf,240454,89.46,2712,52.566667,13.333333
8,Spandau,223962,91.91,2441,52.550000,13.200000
9,Steglitz-Zehlendorf,293989,102.50,2818,52.433333,13.250000


In [11]:
from geopy.geocoders import Nominatim
import folium
from pandas.io.json import json_normalize
import json
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [12]:
address = 'Berlin, DE'

geolocator = Nominatim(user_agent="Berlin_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Berlin are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Berlin are 52.5170365, 13.3888599.


In [13]:
map_Berlin = folium.Map(location=[latitude, longitude], zoom_start=10)

for borough, lat, lng in zip(df['Borough'], df['Latitude'], df['Longitude']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Berlin)  
    
map_Berlin

In [14]:
#Setup Four square API 
CLIENT_ID = 'BHHNZYJ2QTPELHCMLZU5UE2I1BYP3EMKEYK1DPSM5GOBO3H1' # your Foursquare ID
CLIENT_SECRET = 'AY5G3U0ZHTDYOGBL5URRLQ5IKAMNBZK35SYJTWUEJ5RS3FSL' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT = 1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
Berlin_venues = getNearbyVenues(names=df['Borough'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Charlottenburg-Wilmersdorf
Friedrichshain-Kreuzberg
Lichtenberg
Marzahn-Hellersdorf
Mitte
Neukölln
Pankow
Reinickendorf
Spandau
Steglitz-Zehlendorf
Tempelhof-Schöneberg
Treptow-Köpenick


In [17]:
print(Berlin_venues.shape)
Berlin_venues.head()

(171, 7)


,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Charlottenburg-Wilmersdorf,52.5,13.283333,Fressnapf,52.499663,13.288616,Pet Store
1,Charlottenburg-Wilmersdorf,52.5,13.283333,Hotel Kronprinz,52.497091,13.287195,Hotel
2,Charlottenburg-Wilmersdorf,52.5,13.283333,Gleis 1/2,52.500954,13.284037,Platform
3,Charlottenburg-Wilmersdorf,52.5,13.283333,"Reifen-Müller, Georg Müller",52.501168,13.281750,Automotive Shop
4,Charlottenburg-Wilmersdorf,52.5,13.283333,Ditsch,52.500522,13.284126,Bakery


In [18]:
# one hot encoding
Berlin_onehot = pd.get_dummies(Berlin_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Berlin_onehot['Borough'] = Berlin_venues['Borough'] 

# move neighborhood column to the first column
fixed_columns = [Berlin_onehot.columns[-1]] + list(Berlin_onehot.columns[:-1])
Berlin_onehot = Berlin_onehot[fixed_columns]

Berlin_onehot.head()

,Borough,ATM,Art Gallery,Athletics & Sports,Automotive Shop,Bakery,Bank,Bar,Beer Garden,Big Box Store,Bike Shop,Bistro,Boat Rental,Boat or Ferry,Breakfast Spot,Bridge,Bus Stop,Cable Car,Café,Caucasian Restaurant,Climbing Gym,Coffee Shop,Comedy Club,Concert Hall,Convenience Store,Credit Union,Distillery,Doner Restaurant,Drugstore,Eastern European Restaurant,Exhibit,Falafel Restaurant,Fast Food Restaurant,Fried Chicken Joint,Furniture / Home Store,Gas Station,German Restaurant,Gym / Fitness Center,History Museum,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Mexican Restaurant,Middle Eastern Restaurant,Monument / Landmark,Mountain,Museum,Music Venue,Nightclub,Organic Grocery,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Residential Building (Apartment / Condo),Restaurant,River,Rock Club,Scenic Lookout,Schnitzel Restaurant,Shipping Store,Shopping Mall,Smoke Shop,Soccer Field,Spanish Restaurant,Speakeasy,Steakhouse,Supermarket,Sushi Restaurant,Theme Park Ride / Attraction,Track,Tram Station,Trattoria/Osteria,Vietnamese Restaurant
0,Charlottenburg-Wilmersdorf,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Charlottenburg-Wilmersdorf,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Charlottenburg-Wilmersdorf,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Charlottenburg-Wilmersdorf,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Charlottenburg-Wilmersdorf,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [19]:
Berlin_grouped = Berlin_onehot.groupby('Borough').mean().reset_index()
Berlin_grouped

,Borough,ATM,Art Gallery,Athletics & Sports,Automotive Shop,Bakery,Bank,Bar,Beer Garden,Big Box Store,Bike Shop,Bistro,Boat Rental,Boat or Ferry,Breakfast Spot,Bridge,Bus Stop,Cable Car,Café,Caucasian Restaurant,Climbing Gym,Coffee Shop,Comedy Club,Concert Hall,Convenience Store,Credit Union,Distillery,Doner Restaurant,Drugstore,Eastern European Restaurant,Exhibit,Falafel Restaurant,Fast Food Restaurant,Fried Chicken Joint,Furniture / Home Store,Gas Station,German Restaurant,Gym / Fitness Center,History Museum,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Mexican Restaurant,Middle Eastern Restaurant,Monument / Landmark,Mountain,Museum,Music Venue,Nightclub,Organic Grocery,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Residential Building (Apartment / Condo),Restaurant,River,Rock Club,Scenic Lookout,Schnitzel Restaurant,Shipping Store,Shopping Mall,Smoke Shop,Soccer Field,Spanish Restaurant,Speakeasy,Steakhouse,Supermarket,Sushi Restaurant,Theme Park Ride / Attraction,Track,Tram Station,Trattoria/Osteria,Vietnamese Restaurant
0,Charlottenburg-Wilmersdorf,0.00000,0.000000,0.000000,0.20,0.100000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.100000,0.000000,0.000000,0.100000,0.000000,0.00000,0.000000,0.000000,0.000000,0.100000,0.000000,0.00000,0.000000,0.000000,0.000,0.000000,0.00,0.000000,0.0000,0.000000,0.000000,0.000000,0.1,0.00000,0.1,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.100,0.000000,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.1,0.000000,0.00000,0.000,0.000000,0.000000,0.00000,0.000000
1,Friedrichshain-Kreuzberg,0.00000,0.028571,0.028571,0.00,0.028571,0.000000,0.00000,0.00,0.000000,0.028571,0.000000,0.000000,0.000000,0.00,0.028571,0.000000,0.000,0.085714,0.028571,0.00,0.057143,0.000000,0.000000,0.00,0.00000,0.028571,0.000000,0.000000,0.00000,0.028571,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.028571,0.00000,0.057143,0.028571,0.057143,0.028571,0.000000,0.000000,0.00000,0.028571,0.000000,0.000000,0.000000,0.028571,0.00000,0.000000,0.000000,0.000,0.028571,0.00,0.085714,0.0000,0.000000,0.028571,0.000000,0.0,0.00000,0.0,0.00000,0.028571,0.028571,0.028571,0.028571,0.00000,0.000000,0.000000,0.028571,0.000,0.028571,0.000000,0.00000,0.000000,0.000000,0.00,0.028571,0.0,0.000000,0.00000,0.000,0.000000,0.000000,0.00000,0.028571
2,Lichtenberg,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00000,0.000000,0.000000,0.142857,0.00000,0.000000,0.00000,0.142857,0.000000,0.142857,0.000000,0.142857,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000,0.000000,0.00,0.000000,0.0000,0.000000,0.000000,0.000000,0.0,0.00000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.00000,0.000000,0.142857,0.00,0.000000,0.0,0.142857,0.00000,0.000,0.000000,0.142857,0.00000,0.000000
3,Marzahn-Hellersdorf,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.125,0.125000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.125000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.125,0.000000,0.00,0.000000,0.0000,0.0000

In [20]:
num_top_venues = 5

for hood in Berlin_grouped['Borough']:
    print("----"+hood+"----")
    temp = Berlin_grouped[Berlin_grouped['Borough'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Charlottenburg-Wilmersdorf----
             venue  freq
0  Automotive Shop   0.2
1     Liquor Store   0.1
2           Bakery   0.1
3         Platform   0.1
4     Intersection   0.1


----Friedrichshain-Kreuzberg----
            venue  freq
0       Nightclub  0.09
1            Café  0.09
2          Hostel  0.06
3  Ice Cream Shop  0.06
4     Coffee Shop  0.06


----Lichtenberg----
                  venue  freq
0     German Restaurant  0.14
1             Drugstore  0.14
2          Tram Station  0.14
3           Supermarket  0.14
4  Fast Food Restaurant  0.14


----Marzahn-Hellersdorf----
                          venue  freq
0                          Park  0.25
1                          Lake  0.12
2  Theme Park Ride / Attraction  0.12
3                      Mountain  0.12
4                Scenic Lookout  0.12


----Mitte----
          venue  freq
0         Plaza  0.09
1          Park  0.09
2          Lake  0.09
3         Track  0.09
4  Concert Hall  0.09


----Neukölln----
         

In [21]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [22]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Borough_venues_sorted = pd.DataFrame(columns=columns)
Borough_venues_sorted['Borough'] = Berlin_grouped['Borough']

for ind in np.arange(Berlin_grouped.shape[0]):
    Borough_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Berlin_grouped.iloc[ind, :], num_top_venues)

Borough_venues_sorted.head()

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Charlottenburg-Wilmersdorf,Automotive Shop,Intersection,Steakhouse,Pet Store,Scenic Lookout,Liquor Store,Platform,Hotel,Bakery,History Museum
1,Friedrichshain-Kreuzberg,Café,Nightclub,Hostel,Ice Cream Shop,Coffee Shop,Vietnamese Restaurant,Museum,Pool,Plaza,Park
2,Lichtenberg,Furniture / Home Store,Fast Food Restaurant,Tram Station,Supermarket,Drugstore,German Restaurant,Soccer Field,Convenience Store,Credit Union,Distillery
3,Marzahn-Hellersdorf,Park,Theme Park Ride / Attraction,Lake,Café,Cable Car,Scenic Lookout,Mountain,Vietnamese Restaurant,Falafel Restaurant,Distillery
4,Mitte,Concert Hall,Comedy Club,Track,Performing Arts Venue,Plaza,Park,Lake,Outdoor Sculpture,Restaurant,Bistro


In [23]:
# set number of clusters
kclusters = 5

Berlin_grouped_clustering = Berlin_grouped.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Berlin_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([4, 0, 0, 0, 0, 0, 0, 0, 1, 3], dtype=int32)

In [24]:
# add clustering labels
Borough_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Berlin_merged = df

# merge Berlin_grouped with Berlin_data to add latitude/longitude for each Borough
Berlin_merged = Berlin_merged.join(Borough_venues_sorted.set_index('Borough'), on='Borough', how = 'right')

Berlin_merged.head() # check the last columns!

,Borough,Population,Area in km²,Density per km²,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Charlottenburg-Wilmersdorf,319628,64.72,4878,52.500000,13.283333,4,Automotive Shop,Intersection,Steakhouse,Pet Store,Scenic Lookout,Liquor Store,Platform,Hotel,Bakery,History Museum
1,Friedrichshain-Kreuzberg,268225,20.16,13187,52.500000,13.450000,0,Café,Nightclub,Hostel,Ice Cream Shop,Coffee Shop,Vietnamese Restaurant,Museum,Pool,Plaza,Park
2,Lichtenberg,259881,52.29,4952,52.533333,13.500000,0,Furniture / Home Store,Fast Food Restaurant,Tram Station,Supermarket,Drugstore,German Restaurant,Soccer Field,Convenience Store,Credit Union,Distillery
3,Marzahn-Hellersdorf,248264,61.74,4046,52.533333,13.583333,0,Park,Theme Park Ride / Attraction,Lake,Café,Cable Car,Scenic Lookout,Mountain,Vietnamese Restaurant,Falafel Restaurant,Distillery
4,Mitte,332919,39.47,8272,52.516667,13.366667,0,Concert Hall,Comedy Club,Track,Performing Arts Venue,Plaza,Park,Lake,Outdoor Sculpture,Restaurant,Bistro


In [25]:
Berlin_merged.loc[Berlin_merged['Cluster Labels'] == 0, Berlin_merged.columns[[0,1,2,3,4] + list(range(5, Berlin_merged.shape[1]))]]

,Borough,Population,Area in km²,Density per km²,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Friedrichshain-Kreuzberg,268225,20.16,13187,52.500000,13.450000,0,Café,Nightclub,Hostel,Ice Cream Shop,Coffee Shop,Vietnamese Restaurant,Museum,Pool,Plaza,Park
2,Lichtenberg,259881,52.29,4952,52.533333,13.500000,0,Furniture / Home Store,Fast Food Restaurant,Tram Station,Supermarket,Drugstore,German Restaurant,Soccer Field,Convenience Store,Credit Union,Distillery
3,Marzahn-Hellersdorf,248264,61.74,4046,52.533333,13.583333,0,Park,Theme Park Ride / Attraction,Lake,Café,Cable Car,Scenic Lookout,Mountain,Vietnamese Restaurant,Falafel Restaurant,Distillery
4,Mitte,332919,39.47,8272,52.516667,13.366667,0,Concert Hall,Comedy Club,Track,Performing Arts Venue,Plaza,Park,Lake,Outdoor Sculpture,Restaurant,Bistro
5,Neukölln,310283,44.93,6804,52.483333,13.450000,0,Café,Bus Stop,Indian Restaurant,Park,Pub,Music Venue,Climbing Gym,Spanish Restaurant,Bar,Supermarket
6,Pankow,366441,103.01,3476,52.566667,13.400000,0,Café,Organic Grocery,Bakery,Drugstore,Japanese Restaurant,German Restaurant,Middle Eastern Restaurant,Mexican Restaurant,Trattoria/Osteria,Italian Restaurant
7,Reinickendorf,240454,89.46,2712,52.566667,13.333333,0,Bakery,Indian Restaurant,Supermarket,Restaurant,Big Box Store,Bank,Gas Station,Ice Cream Shop,Hostel,History Museum
10,Tempelhof-Schöneberg,335060,53.09,6256,52.466667,13.383333,0,Italian Restaurant,Fried Chicken Joint,Supermarket,Park,Café,Vietnamese Restaurant,Light Rail Station,Bus Stop,Middle Eastern Restaurant,Shipping Store


In [26]:
Berlin_merged.loc[Berlin_merged['Cluster Labels'] == 1, Berlin_merged.columns[[0,1,2,3,4] + list(range(5, Berlin_merged.shape[1]))]]

,Borough,Population,Area in km²,Density per km²,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Spandau,223962,91.91,2441,52.55,13.2,1,Supermarket,Bus Stop,Athletics & Sports,Smoke Shop,Vietnamese Restaurant,Fast Food Restaurant,Credit Union,Distillery,Doner Restaurant,Drugstore


In [27]:
Berlin_merged.loc[Berlin_merged['Cluster Labels'] == 2, Berlin_merged.columns[[0,1,2,3,4] + list(range(5, Berlin_merged.shape[1]))]]

,Borough,Population,Area in km²,Density per km²,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Treptow-Köpenick,241335,168.42,1406,52.45,13.566667,2,Boat or Ferry,Park,Light Rail Station,Supermarket,River,Boat Rental,Vietnamese Restaurant,Credit Union,Distillery,Doner Restaurant


In [28]:
Berlin_merged.loc[Berlin_merged['Cluster Labels'] == 3, Berlin_merged.columns[[0,1,2,3,4] + list(range(5, Berlin_merged.shape[1]))]]

,Borough,Population,Area in km²,Density per km²,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Steglitz-Zehlendorf,293989,102.5,2818,52.433333,13.25,3,Automotive Shop,Café,Eastern European Restaurant,Beer Garden,Vietnamese Restaurant,Fried Chicken Joint,Distillery,Doner Restaurant,Drugstore,Exhibit


In [29]:
Berlin_merged.loc[Berlin_merged['Cluster Labels'] == 4, Berlin_merged.columns[[0,1,2,3,4] + list(range(5, Berlin_merged.shape[1]))]]

,Borough,Population,Area in km²,Density per km²,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Charlottenburg-Wilmersdorf,319628,64.72,4878,52.5,13.283333,4,Automotive Shop,Intersection,Steakhouse,Pet Store,Scenic Lookout,Liquor Store,Platform,Hotel,Bakery,History Museum


**Conclusion**

The data above gives the family the necessary imformation, in conjunction with supplemental information to make a quality decision as to where they should open their restaurant.  More information is needed, such as costs of rent, and other data points which need to be found in order to make a complete decision.  This data above gives the family a narrow field to pick from.